In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
categorical = ['ip','app','channel','device','os']

### data

In [3]:
train_data = pd.read_feather('../data/data/model/train_data.feather', nthreads=15)
train_labels = train_data['is_attributed'].values
train_data = train_data.drop(['is_attributed','day','click_id'], axis=1)

In [4]:
valid_data = pd.read_feather('../data/data/model/valid_data.feather')
valid_labels = valid_data['is_attributed'].values
valid_data = valid_data.drop(['is_attributed','day','click_id'], axis=1)

### model - 1

In [5]:
params = {}
params['label'] = train_labels
params['feature_name'] = list(train_data.columns)
params['categorical_feature'] = categorical
train_matrix = lgb.Dataset(train_data.values, **params)

In [6]:
params = {}
params['label'] = valid_labels
params['feature_name'] = list(valid_data.columns)
params['categorical_feature'] = categorical
valid_matrix = lgb.Dataset(valid_data.values, **params)

In [7]:
booster = {}
booster['boosting_type'] = 'gbdt'
booster['objective'] = 'binary'
booster['learning_rate'] = 0.075
booster['num_leaves'] = 32
booster['max_depth'] = -1
booster['min_child_weight'] = 5
booster['max_bin'] = 255
booster['subsample'] = 0.6
booster['subsample_freq'] = 1
booster['colsample_bytree'] = 0.3
booster['min_split_gain'] = 0
booster['nthread'] = 15
booster['verbose'] = 0
booster['scale_pos_weight'] = 99.7
booster['metric'] = 'auc'

In [8]:
params = {}
params['params'] = booster
params['train_set'] = train_matrix
params['valid_sets'] = [valid_matrix]
params['num_boost_round'] = 1000
params['early_stopping_rounds'] = 50
params['verbose_eval'] = 50

In [9]:
model = lgb.train(**params)

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.973855
[100]	valid_0's auc: 0.980023
[150]	valid_0's auc: 0.98234
[200]	valid_0's auc: 0.982796
[250]	valid_0's auc: 0.983011
[300]	valid_0's auc: 0.983141
[350]	valid_0's auc: 0.983208
[400]	valid_0's auc: 0.98324
[450]	valid_0's auc: 0.983237
Early stopping, best iteration is:
[415]	valid_0's auc: 0.983257


In [10]:
model.save_model('../data/data/model/lightgbm_1.model')

### model - 2

In [5]:
params = {}
params['label'] = train_labels
params['feature_name'] = list(train_data.columns)
params['categorical_feature'] = categorical
train_matrix = lgb.Dataset(train_data.values, **params)

In [6]:
params = {}
params['label'] = valid_labels
params['feature_name'] = list(valid_data.columns)
params['categorical_feature'] = categorical
valid_matrix = lgb.Dataset(valid_data.values, **params)

In [7]:
booster = {}
booster['boosting_type'] = 'gbdt'
booster['objective'] = 'binary'
booster['learning_rate'] = 0.1
booster['num_leaves'] = 24
booster['max_depth'] = -1
booster['min_child_weight'] = 5
booster['max_bin'] = 255
booster['subsample'] = 0.5
booster['subsample_freq'] = 1
booster['colsample_bytree'] = 0.3
booster['min_split_gain'] = 0
booster['nthread'] = 15
booster['verbose'] = 0
booster['scale_pos_weight'] = 99.7
booster['metric'] = 'auc'

In [8]:
params = {}
params['params'] = booster
params['train_set'] = train_matrix
params['valid_sets'] = [valid_matrix]
params['num_boost_round'] = 1000
params['early_stopping_rounds'] = 50
params['verbose_eval'] = 100

In [9]:
model = lgb.train(**params)

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[100]	valid_0's auc: 0.980765
[200]	valid_0's auc: 0.982711
[300]	valid_0's auc: 0.983012
[400]	valid_0's auc: 0.98306
Early stopping, best iteration is:
[381]	valid_0's auc: 0.983067


In [10]:
model.save_model('../data/data/model/lightgbm_2.model')

### feature importance

In [11]:
importance = model.feature_importance(importance_type='gain')
importance = pd.DataFrame(importance, columns=['importance'])
importance['feature'] = list(valid_data.columns)
importance['importance'] = importance['importance'] / importance['importance'].max()
importance = importance[['feature', 'importance']]
importance = importance.sort_values(by='importance', ascending=False)
importance.to_csv('../data/data/model/variable_importance.csv', index=False)

In [12]:
importance.reset_index(drop=True)

,feature,importance
0,app,1.000000
1,channel,0.781995
2,user_count,0.216991
3,ip,0.207361
4,next_click,0.103170
5,ip_app_cnt,0.101527
6,ip_device_cnt,0.082092
7,ip_hour_app_cnt,0.080957
8,app_channel_cnt,0.075539
9,app_channel_rnk,0.048757
